In [5]:
import pandas as pd
import mysql.connector
import sqlalchemy

In [4]:
 cnx = mysql.connector.connect(user='root', password='password', host='127.0.0.1', database='online_store', port='3306')

In [6]:
cur = cnx.cursor()

#### session_id - Идентификатор сессии
#### client_id - Идентификатор клиента
#### src_id - Идентификатор источника с которого пользователь перешел
#### Purchase_flg - Совершалась ли покупка
#### visit_dttm - Дата сессии
#### duration - Время сессии
#### page_count - Кол-во просмотренных страниц
#### last_page_id - Идентификатор последней страницы сессии

In [12]:
cur.execute('''
            CREATE TABLE 
            sessions_info (
                session_id Binary(16) NOT null PRIMARY KEY,
                client_id Binary(16) NOT null,
                src_id Enum('INST', 'VK', 'WEB', 'TG') NOT null,
                Purchase_flg TINYINT(1) NOT null,
                visit_dttm DATETIME NOT null,
                duration TIME NOT null,
                page_count TINYINT UNSIGNED NOT null,	
                last_page_id SMALLINT UNSIGNED NOT null);
            ''')

#### uuid_to_bin(UUID()) - генерирует случайный уникальный UUID и преобразует в BINARY для оптимизации хранения 
#### now() - генерирует текущую дату и время 
#### SEC_TO_TIME(FLOOR(RAND() * 3600)) - генерирует случайное время в формате часы:минуты:секунды 
#### FLOOR(RAND() * 1000) - генерирует случайное целое число от 0 до 999

In [13]:
cur.execute('''
            INSERT INTO sessions_info
            (session_id, client_id, src_id, Purchase_flg, 
            visit_dttm, duration, page_count, last_page_id)
            values 
            (uuid_to_bin(UUID()), uuid_to_bin(UUID()), 'INST', 1, now(),
            SEC_TO_TIME(FLOOR(RAND() * 3600)), FLOOR(RAND() * 255), FLOOR(RAND() * 1000));
            ''')

In [14]:
cur.execute('''
            select * from sessions_info;
            ''')
results = cur.fetchall()
pd.DataFrame(results, columns=['session_id', 'client_id', 'src_id', 'Purchase_flg',
                              'visit_dttm', 'duration', 'page_count', 'last_page_id'])

,session_id,client_id,src_id,Purchase_flg,visit_dttm,duration,page_count,last_page_id
0,"[54, 25, 211, 38, 69, 33, 17, 238, 151, 112, 2...","[54, 25, 224, 170, 69, 33, 17, 238, 151, 112, ...",INST,1,2023-08-27 21:32:23,0 days 00:41:54,7,58


In [18]:
cur.execute('''
            select bin_to_uuid(session_id) from sessions_info;
            ''')
results = cur.fetchall()
pd.DataFrame(results, columns=['session_id'])

,session_id
0,3619d326-4521-11ee-9770-0242ac110002


#### JSON

In [21]:
cur.execute('''
            CREATE TABLE 
            order_items (
                order_items_info json);
            ''')

#### order_id - Идентификатор заказа
#### item_id - Идентификатор товара
#### item_count - Кол-во товара одной позиции
#### price - Цена заказа
#### discount - Размер скидки

In [22]:
cur.execute('''
            INSERT INTO
            order_items
            (order_items_info)
            values
            ('{"order_id": "3619d326-4521-11ee-9770-0242ac110002", "item_id": 4, 
            "item_count": 1, "price": 3000, "discount": 300}');
            ''')

In [23]:
cur.execute('''
            select * from order_items;
            ''')
results = cur.fetchall()
pd.DataFrame(results, columns=['session_id'])

,order_items_info
0,"{""price"": 3000, ""item_id"": 4, ""discount"": 300,..."


In [25]:
cur.execute('''
            select 
            JSON_VALUE(order_items_info , '$.order_id') as order_id,
            JSON_VALUE(order_items_info , '$.item_id') as item_id,
            JSON_VALUE(order_items_info , '$.item_count') as item_count,
            JSON_VALUE(order_items_info , '$.price') as price,
            JSON_VALUE(order_items_info , '$.discount') as discount
            from order_items;
            ''')
results = cur.fetchall()
pd.DataFrame(results, columns=['order_id', 'item_id', 'item_count', 'price', 'discount'])

,order_id,item_id,item_count,price,discount
0,3619d326-4521-11ee-9770-0242ac110002,4,1,3000,300
